# **Dataverse REST API Demo**

In [ ]:
import requests
import json
import uuid
import pandas as pd

In [ ]:
# pandas display options 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## **DATAVERSE**

In [ ]:
DATAVERSE_URL = "https://XXXXXXXXXXXXXXXX.XXXXXX.dynamics.com"

In [ ]:
# developer tools -> networking -> open a dv table -> look for something like a getclientmetadata call, copy bearer token
DATAVERSE_ACCESS_TOKEN = "eyXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

### Read Data Table

In [ ]:
# dataverse table info
dataverse_table_name = "TABLE_NAME"
dataverse_columns = [
    "COLUMN_1",
    "COLUMN_2",
    "COLUMN_3", 
]

In [ ]:
# request session headers 
dataverse_session = requests.Session()
dataverse_session.headers.update(
    {
        "Authorization" : f"Bearer {DATAVERSE_ACCESS_TOKEN}",
        'OData-MaxVersion': '4.0', 
        'OData-Version': '4.0', 
        'If-None-Match': 'null', 
        'Accept': 'application/json',
        "Content-Type": "application/json; charset=utf-8",
        "Prefer" : "odata.maxpagesize=5000",
        "User-Agent" : f"python-requests/{requests.__version__}",
        "Connection" : "keep-alive",
    }
)

In [ ]:
# prep request 
dataverse_columns_comma_seperated = ",".join(dataverse_columns)
dataverse_select_columns = f"?$select={dataverse_columns_comma_seperated}"
dataverse_request_uri = f'{DATAVERSE_URL}/api/data/v9.2/{dataverse_table_name}'
dataverse_request_uri += dataverse_select_columns
print(dataverse_request_uri)

In [ ]:
# request loop
try:
    dataverse_result = dataverse_session.get(dataverse_request_uri)
    dataverse_result_json = json.loads(dataverse_result.content)
except:
    raise Exception(f"{dataverse_result}, {dataverse_result.content}")
dataverse_result_json_value = dataverse_result_json["value"]
print("Items Read: ", len(dataverse_result_json_value), sep="", end="")
while "@odata.nextLink" in dataverse_result_json:
    dataverse_result = dataverse_session.get(dataverse_result_json["@odata.nextLink"])
    dataverse_result_json = json.loads(dataverse_result.content)
    dataverse_result_json_value += dataverse_result_json["value"]
    print("...", len(dataverse_result_json_value), sep="", end="")

In [ ]:
# convert to pandas dataframe 
df_dataverse_raw = pd.DataFrame(dataverse_result_json_value, dtype="object")[dataverse_columns]
df_dataverse_raw.info()

In [ ]:
# view
df_dataverse_raw

### Create New Row

In [ ]:
# request session headers 
dataverse_session = requests.Session()
dataverse_session.headers.update(
    {
        "Authorization" : f"Bearer {DATAVERSE_ACCESS_TOKEN}",
        'OData-MaxVersion': '4.0', 
        'OData-Version': '4.0', 
        'If-None-Match': 'null', 
        'Accept': 'application/json',
        "Content-Type": "application/json; charset=utf-8",
        "User-Agent" : f"python-requests/{requests.__version__}",
        "Prefer" : "return=representation",
    }
)
payload = {
    "COLUMN_1" : "text",
    "COLUMN_2" : "text",
    "COLUMN_3" : 234.23,
}

In [ ]:
# prep request
dataverse_table_name = "TABLE_NAME"
dataverse_request_uri = f'{DATAVERSE_URL}/api/data/v9.2/{dataverse_table_name}'
dataverse_request_uri

In [ ]:
# request 
dataverse_result = dataverse_session.post(
    dataverse_request_uri, 
    data=json.dumps(payload)
)
dataverse_result

### Update a Row

In [ ]:
# request session headers 
dataverse_session = requests.Session()
dataverse_session.headers.update(
    {
        "Authorization" : f"Bearer {DATAVERSE_ACCESS_TOKEN}",
        'OData-MaxVersion': '4.0', 
        'OData-Version': '4.0', 
        'Accept': 'application/json',
        "Content-Type": "application/json; charset=utf-8",
        "User-Agent" : f"python-requests/{requests.__version__}",
        "If-Match" : "*" 
    }
)
payload = {
    "COLUMN_1" : "text",
}

In [ ]:
# prep request 
dataverse_table_name = "TABLE_NAME"
record = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
dataverse_request_uri = f'{DATAVERSE_URL}/api/data/v9.2/{dataverse_table_name}({record})'
dataverse_request_uri

In [ ]:
# request 
dataverse_result = dataverse_session.patch(
    dataverse_request_uri, 
    data=json.dumps(payload)
)
dataverse_result

### Delete a Row

In [ ]:
# request session headers 
dataverse_session = requests.Session()
dataverse_session.headers.update(
    {
        "Authorization" : f"Bearer {DATAVERSE_ACCESS_TOKEN}",
        'OData-MaxVersion': '4.0', 
        'OData-Version': '4.0', 
        'Accept': 'application/json',
        "Content-Type": "application/json; charset=utf-8",
        "User-Agent" : f"python-requests/{requests.__version__}",
    }
)

In [ ]:
# prep request 
dataverse_table_name = "TABLE_NAME"
record = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
dataverse_request_uri = f'{DATAVERSE_URL}/api/data/v9.2/{dataverse_table_name}({record})'
dataverse_request_uri

In [ ]:
# request 
dataverse_result = dataverse_session.delete(
    dataverse_request_uri, 
    data=json.dumps(payload)
)
dataverse_result